In [86]:
import pandas as pd


In [87]:
pd.__version__

'2.2.3'

In [88]:
jan23 = pd.read_parquet("yellow_tripdata_2023-01.parquet")
feb23 = pd.read_parquet("yellow_tripdata_2023-02.parquet")

#### Q1: downloading the data

We'll use [the same NYC taxi dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page), but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".
Download the data for January and February 2023.
Read the data for January. How many columns are there?

In [89]:
jan23.columns, len(jan23.columns)

(Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
        'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
        'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
        'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
        'total_amount', 'congestion_surcharge', 'airport_fee'],
       dtype='object'),
 19)

In [90]:
jan23.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


#### Q2: computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes.
What's the standard deviation of the trips duration in January?

In [91]:
jan23["duration"] = jan23["tpep_dropoff_datetime"] - jan23["tpep_pickup_datetime"]
jan23["duration"] = jan23["duration"].dt.seconds / 60.0

In [92]:
jan23["duration"].std()

np.float64(41.62919110966266)

#### Q3: dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).
What fraction of the records left after you dropped the outliers?

In [93]:
rows_before = jan23.shape[0]

In [94]:
jan23 = jan23[(jan23["duration"] >= 1) & (jan23["duration"] <= 60)]

In [95]:
jan23.shape[0] / rows_before

0.9812212604417814

#### Q4: one-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
- Fit a dictionary vectorizer
- Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [96]:
cols_before = jan23.shape[1]

In [97]:
prefix = "ftr"

In [98]:
jan23 = pd.get_dummies(data=jan23, columns=["PULocationID", "DOLocationID"], prefix=[prefix, prefix], dtype=int)

MemoryError: Unable to allocate 5.72 GiB for an array with shape (255, 3009176) and data type int64

In [ ]:
jan23.shape[1] - cols_before

513

#### Q5: training a model

Now let's use the feature matrix from the previous step to train a model.

- Train a plain linear regression model with default parameters, where duration is the response variable
- Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr_cols = [x for x in jan23.columns if x.startswith(prefix) or x == "duration"]

In [ ]:
lr_cols

['duration',
 'ftr_1',
 'ftr_2',
 'ftr_3',
 'ftr_4',
 'ftr_5',
 'ftr_6',
 'ftr_7',
 'ftr_8',
 'ftr_9',
 'ftr_10',
 'ftr_11',
 'ftr_12',
 'ftr_13',
 'ftr_14',
 'ftr_15',
 'ftr_16',
 'ftr_17',
 'ftr_18',
 'ftr_19',
 'ftr_20',
 'ftr_21',
 'ftr_22',
 'ftr_23',
 'ftr_24',
 'ftr_25',
 'ftr_26',
 'ftr_28',
 'ftr_29',
 'ftr_30',
 'ftr_31',
 'ftr_32',
 'ftr_33',
 'ftr_34',
 'ftr_35',
 'ftr_36',
 'ftr_37',
 'ftr_38',
 'ftr_39',
 'ftr_40',
 'ftr_41',
 'ftr_42',
 'ftr_43',
 'ftr_44',
 'ftr_45',
 'ftr_46',
 'ftr_47',
 'ftr_48',
 'ftr_49',
 'ftr_50',
 'ftr_51',
 'ftr_52',
 'ftr_53',
 'ftr_54',
 'ftr_55',
 'ftr_56',
 'ftr_57',
 'ftr_58',
 'ftr_60',
 'ftr_61',
 'ftr_62',
 'ftr_63',
 'ftr_64',
 'ftr_65',
 'ftr_66',
 'ftr_67',
 'ftr_68',
 'ftr_69',
 'ftr_70',
 'ftr_71',
 'ftr_72',
 'ftr_73',
 'ftr_74',
 'ftr_75',
 'ftr_76',
 'ftr_77',
 'ftr_78',
 'ftr_79',
 'ftr_80',
 'ftr_81',
 'ftr_82',
 'ftr_83',
 'ftr_85',
 'ftr_86',
 'ftr_87',
 'ftr_88',
 'ftr_89',
 'ftr_90',
 'ftr_91',
 'ftr_92',
 'ftr_93',
 'ftr_

In [ ]:
jan23 = jan23[lr_cols]

In [ ]:
lr = LinearRegression()
X_train = jan23.drop(columns="duration")
y_train = jan23["duration"]

In [ ]:
lr.fit(X_train, y_train)

MemoryError: Unable to allocate 5.72 GiB for an array with shape (255, 3009176) and data type object

In [ ]:
from sklearn.metrics import root_mean_squared_error

In [ ]:
root_mean_squared_error(y_true=y_train, y_pred=lr.predict(X_train))

#### Q6: evaluating the model

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [ ]:
# calculate duration
feb23["duration"] = feb23["tpep_dropoff_datetime"] - feb23["tpep_pickup_datetime"]
feb23["duration"] = feb23["duration"].dt.seconds / 60.0

In [ ]:
# clear outliers
feb23 = feb23[(feb23["duration"] >= 1) & (feb23["duration"] <= 60)]

In [ ]:
# one-hot encode
feb23 = pd.get_dummies(data=feb23, columns=["PULocationID", "DOLocationID"], prefix=[prefix, prefix], dtype=int)

In [ ]:
# project columns
lr_cols = [x for x in jan23.columns if x.startswith(prefix) or x == "duration"]
feb23 = feb23[lr_cols]

In [ ]:
X_valid = feb23.drop("duration")
y_valid = feb23["duration"]

In [ ]:
root_mean_squared_error(y_true=y_valid, y_pred=lr.predict(X_valid))